# Introduction to SQL/3

## Set Operations

### SQL Set Operations on Courses by Semester

**Find courses that ran in Fall 2009 or in Spring 2010:**

```sql
SELECT course_id FROM section WHERE sem = 'Fall' AND year = 2009
UNION
SELECT course_id FROM section WHERE sem = 'Spring' AND year = 2010;
```

**Find courses that ran in Fall 2009 and in Spring 2010:**

```sql
SELECT course_id FROM section WHERE sem = 'Fall' AND year = 2009
INTERSECT
SELECT course_id FROM section WHERE sem = 'Spring' AND year = 2010;
```

**Find courses that ran in Fall 2009 but not in Spring 2010:**

```sql
SELECT course_id FROM section WHERE sem = 'Fall' AND year = 2009
EXCEPT
SELECT course_id FROM section WHERE sem = 'Spring' AND year = 2010;
```

### SQL Queries to Analyze Instructor Salaries

**1. Find the salaries of all instructors that are less than the largest salary:**

```sql
SELECT DISTINCT T.salary
FROM instructor AS T, instructor AS S
WHERE T.salary < S.salary;
```

**2. Find all the salaries of all instructors:**

```sql
SELECT DISTINCT salary
FROM instructor;
```

**3. Find the largest salary of all instructors:**

```sql
(SELECT DISTINCT salary FROM instructor)
EXCEPT
(SELECT DISTINCT T.salary
 FROM instructor AS T, instructor AS S
 WHERE T.salary < S.salary);
```

### Set Operations: `UNION`, `INTERSECT`, and `EXCEPT`

- Each of the above operations **automatically eliminates duplicates**.

- To **retain all duplicates**, use the corresponding **multiset versions**:
  - `UNION ALL`
  - `INTERSECT ALL`
  - `EXCEPT ALL`

---

### Behavior with Duplicate Tuples

Suppose a tuple occurs `m` times in relation `r` and `n` times in relation `s`, then:

- It occurs **`m + n` times** in:  
    ```sql
    r UNION ALL s
    ```

- It occurs **min(m, n) times** in:
    ```sql
    r INTERSECT ALL s
    ```

- It occurs **max(0, m - n) times** in:
    ```sql
    r EXCEPT ALL s
    ```

## Null Values

### Working with `NULL` in SQL

- It is possible for tuples to have a **null value**, denoted by `NULL`, for some of their attributes.
- `NULL` signifies an **unknown value** or that a **value does not exist**.
- The result of any arithmetic expression involving `NULL` is also `NULL`.

**Example:**
```sql
5 + NULL  -- returns NULL
```

- Checking for **NULL** Values
    - The predicate **IS NULL** can be used to check for null values.

- Example: Find all instructors whose salary is null:
    ```sql
    SELECT name
    FROM instructor
    WHERE salary IS NULL;
    ```

- It is not possible to test for NULL values using standard comparison operators (like =, <, >).
- Instead, use:
    - **IS NULL**
    - **IS NOT NULL**

### Three-Valued Logic in SQL: `TRUE`, `FALSE`, and `UNKNOWN`

- SQL uses **three logical values**: `TRUE`, `FALSE`, and `UNKNOWN`.

- Any **comparison with `NULL`** returns `UNKNOWN`.

**Examples:**
```sql
5 < NULL        -- returns UNKNOWN
NULL <> NULL    -- returns UNKNOWN
NULL = NULL     -- returns UNKNOWN
```

#### Truth Table Behavior with UNKNOWN
- `OR` Operator
    - `UNKNOWN OR TRUE → TRUE`
    - `UNKNOWN OR FALSE → UNKNOWN`
    - `UNKNOWN OR UNKNOWN → UNKNOWN`
- `AND` Operator
    - `TRUE AND UNKNOWN → UNKNOWN`
    - `FALSE AND UNKNOWN → FALSE`
    - `UNKNOWN AND UNKNOWN → UNKNOWN`
- `NOT` Operator
    - `NOT UNKNOWN → UNKNOWN`

#### Interpreting UNKNOWN in WHERE Clauses
- If a predicate in the WHERE clause evaluates to UNKNOWN, it is treated as FALSE.
- This means such rows are excluded from the result.

- Note: To explicitly check if a predicate evaluates to UNKNOWN, use:

    ```sql
    P IS UNKNOWN
    ```

- This returns TRUE if the predicate P evaluates to UNKNOWN.

## Aggregate Functions

- These functions operate on the multiset of values of a column of a relation, and return a value
    - `avg`: average value
    - `min`: minimum value
    - `max`: maximum value
    - `sum`: sum of values
    - `count`: number of values

### SQL Aggregate Queries

#### 1. Find the average salary of instructors in the Computer Science department:
```sql
SELECT AVG(salary)
FROM instructor
WHERE dept_name = 'Comp. Sci';
```
#### 2. Find the total number of instructors who teach a course in the Spring 2010 semester:
```sql
SELECT COUNT(DISTINCT ID)
FROM teaches
WHERE semester = 'Spring' AND year = 2010;
```
#### 3. Find the number of tuples in the course relation:
```sql
SELECT COUNT(*)
FROM courses;
```

### Using `GROUP BY` in SQL

The `GROUP BY` clause is used in SQL to group rows that have the same values in specified columns into summary rows. Commonly used with aggregate functions like `COUNT`, `SUM`, `AVG`, `MAX`, or `MIN`.

#### Example:
**Find the average salary of instructors in each department**

#### Input Table: `instructor`

| ID   | name     | dept_name     | salary  |
|------|----------|---------------|---------|
| 101  | Alice    | Comp. Sci     | 90000   |
| 102  | Bob      | Comp. Sci     | 85000   |
| 103  | Charlie  | Mathematics   | 70000   |
| 104  | Dave     | Physics       | 80000   |
| 105  | Eve      | Physics       | 85000   |

```sql
SELECT dept_name, AVG(salary) AS avg_salary
FROM instructor
GROUP BY dept_name;
```

#### Output Table:

| dept\_name  | avg\_salary |
| ----------- | ----------- |
| Comp. Sci   | 87500       |
| Mathematics | 70000       |
| Physics     | 82500       |

---

- Attributes in **select** clause outside of aggregate functions must appear in **group by** list
    ```sql
    /* erroneous query */
    select dept_name, ID, avg(salary)
    from instructor
    group by dept_name;


### Using `Having` in SQL

- Find the names and average salaries of all departments whose average salary is greater than 42000
    ```sql
    select dept_name, ID, avg(salary)
    from instructor
    group by dept_name
    having avg(salary) > 42000;
    ```
- Note: predicates in the `having` clause are applied after the formation of groups whereas predicates in the `where` clause are applied before forming groups

### Null Values and Aggregates

- Total all salaries
    ```sql
    select sum (salary)
    from instructor;
    ```
    - Above statement ignores null amounts
    - Result is null if there is no non-null amount
- All aggregate operations except `count(*)` ignore tuples with null values on the aggregated attributes
- What if collection has only null values?
    - count returns 0
    - all other aggregates return null